In [ ]:
# Create the training dataset
from sklearn.datasets import load_diabetes
import pandas as pd

features, target = load_diabetes(return_X_y=True)
diabetes_df = pd.DataFrame(features)
diabetes_df['target'] = target

In [ ]:
# Store and register as dataset
from azureml.core import Workspace, Dataset

ws = Workspace.from_config()
dstore = ws.get_default_datastore()
ds = Dataset.Tabular.register_pandas_dataframe(
    dataframe=diabetes_df,
    target=(dstore,"/samples/diabetes"),
    name="diabetes",
    description="The sklearn diabetes dataset")

In [ ]:
# Get the computation target for training
compute_target = ws.compute_targets["cpu-cluster"]

In [ ]:
# Create automl experiment
from azureml.train.automl import AutoMLConfig

# Split dataset
train_ds,validate_ds = ds.random_split(percentage=0.8, seed=1337)

experiment_config = AutoMLConfig(
    validation_data = validate_ds,
    training_data = train_ds,
    label_column_name = "target",
    task = "regression",
    compute_target = compute_target,
    primary_metric = 'r2_score',
    experiment_timeout_hours = 0.25,
    model_explainability = True  # This is the default value
)

In [ ]:
# Let it train
from azureml.core.experiment import Experiment

my_experiment = Experiment(ws, 'auto-ml-deployment-experiment')
run = my_experiment.submit(experiment_config, show_output=True)